In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 41.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=37344f4d0b82842d61cb32123b18dc8b635152184925c85afc74573a17e2457f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import udf, col, monotonically_increasing_id
from pyspark.sql.types import *
from pyspark.ml.feature import QuantileDiscretizer
spark = SparkSession.builder.appName("Movies").master("local[*]").getOrCreate()
spark

In [ ]:
movie = spark.read.csv("/content/drive/MyDrive/Mentorias/ID-Eric/Dataset Peliculas/movie.csv", header = True)
gender = spark.read.csv("/content/drive/MyDrive/Mentorias/ID-Eric/Dataset Peliculas/gender.csv", header = True)
actor = spark.read.csv("/content/drive/MyDrive/Mentorias/ID-Eric/Dataset Peliculas/actor.csv", header = True)
ma = spark.read.csv("/content/drive/MyDrive/Mentorias/ID-Eric/Dataset Peliculas/ma.csv", header = True)

In [ ]:
movie.show()

+---+------+--------------------+--------+--------+-----------------+------------+---------------+------+
|_c0|mov_id|           mov_title|mov_year|mov_time|         mov_lang|  mov_dt_rel|mov_rel_country|genero|
+---+------+--------------------+--------+--------+-----------------+------------+---------------+------+
|  0|   901| Vertigo         ...|    1958|     128| English         | 1958-08-24 |             UK|  1010|
|  1|   902| The Innocents   ...|    1961|     100| English         | 1962-02-19 |             SW|  1008|
|  2|   903| Lawrence of Arab...|    1962|     216| English         | 1962-12-11 |             UK|  1002|
|  3|   904| The Deer Hunter ...|    1978|     183| English         | 1979-03-08 |             UK|  1013|
|  4|   905| Amadeus         ...|    1984|     160| English         | 1985-01-07 |             UK|  1004|
|  5|   906| Blade Runner    ...|    1982|     117| English         | 1982-09-09 |             UK|  1012|
|  6|   907| Eyes Wide Shut  ...|    1999|    

In [ ]:
actor.show()

+---+------+--------------------+--------------------+----------+
|_c0|act_id|           act_fname|           act_lname|act_gender|
+---+------+--------------------+--------------------+----------+
|  0|   101| James           ...| Stewart         ...|         M|
|  1|   102| Deborah         ...| Kerr            ...|         F|
|  2|   103| Peter           ...| OToole          ...|         M|
|  3|   104| Robert          ...| De Niro         ...|         M|
|  4|   105| F. Murray       ...| Abraham         ...|         M|
|  5|   106| Harrison        ...| Ford            ...|         M|
|  6|   107| Nicole          ...| Kidman          ...|         F|
|  7|   108| Stephen         ...| Baldwin         ...|         M|
|  8|   109| Jack            ...| Nicholson       ...|         M|
|  9|   110| Mark            ...| Wahlberg        ...|         M|
| 10|   111| Woody           ...| Allen           ...|         M|
| 11|   112| Claire          ...| Danes           ...|         F|
| 12|   11

In [ ]:
gender.show()

+---+------+----------+
|_c0|gen_id| gen_title|
+---+------+----------+
|  0|  1001|    Action|
|  1|  1002| Adventure|
|  2|  1003| Animation|
|  3|  1004| Biography|
|  4|  1005|    Comedy|
|  5|  1006|     Crime|
|  6|  1007|     Drama|
|  7|  1008|    Horror|
|  8|  1009|     Music|
|  9|  1010|   Mystery|
| 10|  1011|   Romance|
| 11|  1012|  Thriller|
| 12|  1013|       War|
+---+------+----------+



In [ ]:
ma.show()

+---+------+------+--------------------+
|_c0|act_id|mov_id|                role|
+---+------+------+--------------------+
|  0|   101|   901| John Scottie Fer...|
|  1|   102|   902|        Miss Giddens|
|  2|   103|   903|       T.E. Lawrence|
|  3|   104|   904|             Michael|
|  4|   105|   905|     Antonio Salieri|
|  5|   106|   906|        Rick Deckard|
|  6|   107|   907|       Alice Harford|
|  7|   108|   908|             McManus|
|  8|   110|   910|         Eddie Adams|
|  9|   111|   911|         Alvy Singer|
| 10|   112|   912|                 San|
| 11|   113|   913|       Andy Dufresne|
| 12|   114|   914|      Lester Burnham|
| 13|   115|   915| Rose DeWitt Bukater|
| 14|   116|   916|        Sean Maguire|
| 15|   117|   917|                  Ed|
| 16|   118|   918|              Renton|
| 17|   120|   920|     Elizabeth Darko|
| 18|   121|   921|         Older Jamal|
| 19|   122|   922|              Ripley|
+---+------+------+--------------------+
only showing top

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS Movies")

DataFrame[]

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS Movies")

In [ ]:
spark.sql("USE Movies")

DataFrame[]

In [ ]:
movie.write.format("parquet").mode("overwrite").saveAsTable("Movies.Movie")

In [ ]:
actor.write.format("parquet").mode("overwrite").saveAsTable("Movies.Actor")

In [ ]:
gender.write.format("parquet").mode("overwrite").saveAsTable("Movies.Gender")

In [ ]:
ma.write.format("parquet").mode("overwrite").saveAsTable("Movies.Movie_Actor")

In [ ]:
spark.sql("SELECT * FROM Movie").show()

+---+------+--------------------+--------+--------+-----------------+------------+---------------+------+
|_c0|mov_id|           mov_title|mov_year|mov_time|         mov_lang|  mov_dt_rel|mov_rel_country|genero|
+---+------+--------------------+--------+--------+-----------------+------------+---------------+------+
|  0|   901| Vertigo         ...|    1958|     128| English         | 1958-08-24 |             UK|  1010|
|  1|   902| The Innocents   ...|    1961|     100| English         | 1962-02-19 |             SW|  1008|
|  2|   903| Lawrence of Arab...|    1962|     216| English         | 1962-12-11 |             UK|  1002|
|  3|   904| The Deer Hunter ...|    1978|     183| English         | 1979-03-08 |             UK|  1013|
|  4|   905| Amadeus         ...|    1984|     160| English         | 1985-01-07 |             UK|  1004|
|  5|   906| Blade Runner    ...|    1982|     117| English         | 1982-09-09 |             UK|  1012|
|  6|   907| Eyes Wide Shut  ...|    1999|    

In [ ]:
# Nombre del género de las películas que se estrenaron en USA
spark.sql("""

           SELECT t1.mov_title, t2.gen_title FROM (SELECT mov_title, genero FROM Movie WHERE mov_rel_country = 'USA') AS t1
                  INNER JOIN Gender AS t2 ON t1.genero = t2.gen_id

          """).show()

+--------------------+---------+
|           mov_title|gen_title|
+--------------------+---------+
| Annie Hall      ...|   Comedy|
+--------------------+---------+

